In [2]:
# | default_exp tools.debate

In [ ]:
# | export
from typing import Dict, Any
from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory
import asyncio

class DebateTool(BaseTool):
    """Tool wrapper for DebateAgent"""
    
    def __init__(self):
        super().__init__(ToolMetadata(
            name="debate",
            description="Multi-perspective analysis for comparison questions and decision making",
            category=ToolCategory.INTELLIGENCE
        ))
    
    def execute(self, topic: str, context: str = "", max_rounds: int = 2) -> Dict[str, Any]:
        """Execute debate analysis using DebateAgent"""
        try:
            from agentic.agent.debater import create_debate
            
            # Check if there's already a running event loop
            try:
                loop = asyncio.get_running_loop()
                # If we're in an async context, create a task
                import concurrent.futures
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    future = executor.submit(self._run_debate_sync, topic, context, max_rounds)
                    result = future.result()
                return {"success": True, "result": result}
            except RuntimeError:
                # No running loop, create a new one
                result = asyncio.run(create_debate(
                    topic=topic,
                    context=context,
                    max_rounds=max_rounds
                ))
                return {"success": True, "result": result}
                
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    def _run_debate_sync(self, topic: str, context: str, max_rounds: int):
        """Run debate in a separate thread with its own event loop"""
        from agentic.agent.debater import create_debate
        return asyncio.run(create_debate(
            topic=topic,
            context=context,
            max_rounds=max_rounds
        ))
    
    def get_parameters_schema(self) -> Dict[str, Any]:
        return {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic or question to analyze"
                },
                "context": {
                    "type": "string", 
                    "description": "Additional context for the analysis",
                    "default": ""
                },
                "max_rounds": {
                    "type": "integer",
                    "description": "Maximum number of debate rounds",
                    "default": 2
                }
            },
            "required": ["topic"]
        }

